In [1]:
import numpy as np
import pandas as pd
from simKNN import SimKNN

In [2]:
%%time
song = pd.read_json("res/song_meta.json")[["id", "song_name", "artist_id_basket", "artist_name_basket",\
                                            "album_id", "album_name", "song_gn_gnr_basket",\
                                            "song_gn_dtl_gnr_basket", "issue_date"]]
# genre = pd.read_json("res/genre_gn_all.json", typ="series")
train = pd.read_json("res/train.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
# val = pd.read_json("res/val.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
test = pd.read_json("res/test.json")[["id", "plylst_title", "songs", "tags", "like_cnt", "updt_date"]]
# display(song, genre, train, val, test)

CPU times: user 10.6 s, sys: 1.01 s, total: 11.6 s
Wall time: 11.7 s


In [3]:
data, X = train[["id", "songs"]], test[["id", "songs"]]
display(data, X)

,id,songs
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954..."
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427..."
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598..."
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033..."
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,..."
...,...,...
115066,120325,"[429629, 441511, 612106, 516359, 691768, 38714..."
115067,106976,"[321330, 216057, 534472, 240306, 331098, 23288..."
115068,11343,"[50512, 249024, 250608, 371171, 229942, 694943..."
115069,131982,"[533534, 608114, 343608, 417140, 609009, 30217..."


,id,songs
0,70107,"[398985, 449403, 411543, 528044, 143048, 98020..."
1,7461,"[196298, 269984, 267805, 175867, 529244, 63825..."
2,90348,"[273433, 331003, 68432, 411659, 117793, 616860..."
3,58617,"[702227, 48152, 440008, 358488, 701041, 540721..."
4,102395,"[630683, 481582, 528550, 285114, 506667, 17922..."
...,...,...
10735,137930,"[323755, 397594, 445908, 570242, 221853, 20018..."
10736,936,"[105140, 582252, 199262, 422915, 547967, 48791..."
10737,110589,"[21976, 207746, 40025, 31635, 567462, 641799, ..."
10738,2605,"[234554, 265033, 507260, 83092, 366757, 497097..."


In [4]:
# FIXME
def pred_info(u, track_ids, relevances, song=song, test=test):
    pd.set_option('display.max_rows', None)
    plylst_id = test.id[u]
    _track_ids = test.songs[u]
    print(plylst_id)
    pred = pd.DataFrame(song.song_name[track_ids], columns=["id", "song_name"])
    original = pd.DataFrame(song.song_name[_track_ids], columns=["id", "song_name"])
    display(pd.concat([pred, original], axis=1, ignore_index=True))

In [5]:
simknn = SimKNN(k=100, metric="idf")
simknn.fit(data)
pred = simknn.predict(X, limit=1)
print(pred)


[(0, array([227169, 305064,  45581, 695910, 387621, 472395, 254896, 331003,
       628459, 157389, 688951, 513270, 524287, 272924, 275414, 477609,
       474175, 534493, 489157, 395438, 622533, 187991, 544625, 179696,
       448738, 234958, 267224, 256956, 618150,  89571, 635759,  75643,
       311240,  72222, 228501, 365722, 602996, 631627, 374617, 286409,
       395511, 477905, 173798, 388635, 163560, 372235, 248226, 407073,
       609622, 426348,  54408, 190352, 225426, 455668, 130494, 482972,
        23520, 475696, 110904, 519630, 595353, 612698, 257577, 518930,
       254801, 298203, 684431, 561919, 424712, 331485, 351553, 154941,
       549303, 602247, 112403, 582223, 252700, 511300, 151412, 616860,
       640958, 555134, 656512, 591035, 144826, 209136, 671533, 307012,
        41543,  26627, 446477, 407044, 256688, 577817, 705682, 549815,
       632755, 297010, 378627, 401226]), array([0.21451366, 0.19908749, 0.19535191, 0.18432098, 0.14856744,
       0.14852539, 0.14665009, 0.1

In [7]:
simknn2 = SimKNN(k=100)
simknn2.fit(data)
pred2 = simknn2.predict(X, limit=1)
print(pred2)


[(0, array([374617, 688951, 305064, 566257,  45581, 635759, 249594, 524287,
        80103, 227169, 228501, 579592, 275414, 544625, 477609, 163560,
       387621, 272924,  24221, 254896, 331003,  15958, 307012, 232305,
       318871, 609247, 695910, 472395, 297010, 631627, 225426, 144826,
       465471, 513270, 232224, 474175, 157389, 395438, 489157, 622533,
       187991, 534493, 329634, 541219, 602629, 609622, 138031, 388635,
       628459, 120626, 293024, 115765,   7749, 477905, 371829, 101388,
       656512, 445084, 553894,   8763, 543751, 637303, 234958,  37235,
         1546, 311834, 185513, 311240, 555134, 130275, 679405, 109190,
        15846, 632755, 407044, 284696, 644455, 425758, 618150, 388487,
       413823, 274716,   7837, 303268, 254801, 405542, 155016, 407073,
        54408, 481045,  89571, 386822, 302604, 148788, 221986, 703596,
       181369, 264701, 237407, 486768]), array([0.25800223, 0.23646837, 0.21265428, 0.19829843, 0.19353048,
       0.17944714, 0.16766213, 0.1

In [21]:
u, t1, rel1 = pred[0]
u2, t2, rel2 = pred2[0]
_track_ids = test.songs[u]

In [25]:
common = np.intersect1d(t1, t2)
common

array([ 45581,  54408,  89571, 144826, 157389, 163560, 187991, 225426,
       227169, 228501, 234958, 254801, 254896, 272924, 275414, 297010,
       305064, 307012, 311240, 331003, 374617, 387621, 388635, 395438,
       407044, 407073, 472395, 474175, 477609, 477905, 489157, 513270,
       524287, 534493, 544625, 555134, 609622, 618150, 622533, 628459,
       631627, 632755, 635759, 656512, 688951, 695910])

In [34]:
c1 = [np.where(t1 == i)[0][0] for i in common]
c2 = [np.where(t2 == i)[0][0] for i in common]
print("from cosine to idf")
for i, v in enumerate(common):
    if c2[i] > c1[i]:
        change = "up"
    elif c2[i] < c1[i]:
        change = "down"
    else:
        change = "same"
    print("{}: {} -> {} // {} // diff={}".format(v, c2[i], c1[i], change, c2[i] - c1[i]))

from cosine to idf
45581: 4 -> 2 // up // diff=2
54408: 88 -> 50 // up // diff=38
89571: 90 -> 29 // up // diff=61
144826: 31 -> 84 // down // diff=-53
157389: 36 -> 9 // up // diff=27
163560: 15 -> 44 // down // diff=-29
187991: 40 -> 21 // up // diff=19
225426: 30 -> 52 // down // diff=-22
227169: 9 -> 0 // up // diff=9
228501: 10 -> 34 // down // diff=-24
234958: 62 -> 25 // up // diff=37
254801: 84 -> 64 // up // diff=20
254896: 19 -> 6 // up // diff=13
272924: 17 -> 13 // up // diff=4
275414: 12 -> 14 // down // diff=-2
297010: 28 -> 97 // down // diff=-69
305064: 2 -> 1 // up // diff=1
307012: 22 -> 87 // down // diff=-65
311240: 67 -> 32 // up // diff=35
331003: 20 -> 7 // up // diff=13
374617: 0 -> 38 // down // diff=-38
387621: 16 -> 4 // up // diff=12
388635: 47 -> 43 // up // diff=4
395438: 37 -> 19 // up // diff=18
407044: 74 -> 91 // down // diff=-17
407073: 87 -> 47 // up // diff=40
472395: 27 -> 5 // up // diff=22
474175: 35 -> 16 // up // diff=19
477609: 14 -> 15 // dow

In [43]:
for t in t1:
    print(song[song.id == t].song_gn_gnr_basket.values)

[list(['GN2500', 'GN0200'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN2500', 'GN0200'])]
[list(['GN2500', 'GN0200'])]
[list(['GN1500'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0200'])]
[list(['GN0200'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0300'])]
[list(['GN0200'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN0100'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0300'])]
[list(['GN0500', 'GN0300'])]
[list(['GN0100'])]
[list(['GN0300'])]
[list(['GN1900'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0400', 'GN2500'])]
[list(['GN2500', 'GN0200'])]
[list(['GN2500', 'GN0100'])]
[list(['GN1200'])]
[list(['GN2500', 'GN0100'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN2500', 'GN0100'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0900'])]
[list(['GN0300'])]
[list(['GN0300'])]
[list(['GN2500', 'GN0200'])]
[list(['GN0500', 'GN0300'])]
[list(['GN0300'])]
[list(['GN2600'])]
[list(['GN0100'])]
[list(['GN0400'])]
[list(['GN0500', 'GN0800'])]
[list(['GN250

In [7]:
# pd.set_option('display.max_rows', None)
u, track_ids, relevances = pred[0]
_track_ids = test.songs[u]
print(song.song_name[track_ids])
print(song.song_name[_track_ids])

227169                                          이럴 줄 알았어
305064                                 Solo (Feat. Alex)
45581                                                답답해
695910                                            내가 아니야
387621                                      I Don`t Care
472395                                              미운사람
254896                                          Hate You
331003                                 사랑과 전쟁 (Narr. 하하)
628459                                   있잖아 (Rock Ver.)
157389                                              미치겠어
688951              ? (물음표) (Feat. 최자 Of 다이나믹듀오, Zion.T)
513270                                               Boo
524287                     Love (Feat. Bumkey, Paloalto)
272924                            Gone (Feat. 주희 Of 에이트)
275414                            Officially Missing You
477609                                             사고쳤어요
474175                                            Baby U
534493          시간이 날 기다려 (Tik 

In [ ]:
%%time
u, track_ids, relevances = next(pred)
# pred_info(u, track_ids, relevances)
_track_ids = test.songs[u]
song.song_name[track_ids], song.song_name[_track_ids]
# display 해주는 함수 만들기
# 겹치는 노래 제외해야 함
# html

In [12]:
size = int(data.shape[0] * (1 - 0.3))
x = data.iloc[size:, :]
x.songs[:]

[112418, 363240, 75991, 386495, 22061]